Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import umap
import bokeh.layouts as bkl
import bokeh.plotting as bk
import bokeh.transform as btr
from io import BytesIO
import base64
from itertools import zip_longest, cycle
from bokeh.models import *
from bokeh.palettes import *
from utils.dawnfrb import get_frb_info, tns_names

theme = 'rainbow'

/home/guutz/radioconda/envs/umap/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/guutz/radioconda/envs/umap/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/guutz/radioconda/envs/umap/lib/python3.9/site-packages/umap/distances.

Get Data

In [2]:
timeSeries = get_frb_info('ts')
pad_len = max([len(x) for x in timeSeries])
padded_timeSeries = [np.concatenate([x, np.zeros(pad_len-len(x))]) for x in timeSeries]
timeSeriesFFT_real = np.fft.rfft(padded_timeSeries).real

displayProperties = {
    'timeSeriesFFT': {
        'data': timeSeriesFFT_real,
        'graph': True,
        'umap': {
            'n_neighbors': 10,
            'min_dist': 0.1,
            'metric': 'euclidean',
            'verbose': True,
        }
    },
    'timeSeries': {
        'data': padded_timeSeries,
        'graph': True,
    },
    'tns_name': {
        'data': tns_names,
        'tooltip': True,
    },
    'dm': {
        'data': get_frb_info('dm'),
        'color': True,
    },
    'width_ms': {
        'data': [i*1000 for i in get_frb_info('width_fitb')],
        'color': True,
        'tooltip': True,
    },
    'n_subbursts': {
        'data': get_frb_info('n_subbursts'),
        'color': True,
    },
    'repeater': {
        'data': ["✔️" if rp != -9999.0 else "❌" for rp in get_frb_info('repeater_name')],
        'color': True,
    },
    'scatterfit': {
        'data': ["✔️" if sf else "❌" for sf in get_frb_info('scatterfit')],
        'color': True,
    },
}

Set up dataframe

In [4]:
PLOT_DF = pd.DataFrame()
_to_graph = []
_tooltips = []
_color_options = []
_embedding_options = []

for key, value in displayProperties.items():
    if value.get('tooltip') or value.get('color'):
        PLOT_DF[key] = value['data']
    if value.get('tooltip'):
        _tooltips.append(key)
    if value.get('graph'):
        _to_graph.append(key)
    if value.get('color'):
        _color_options.append(key)
    if value.get('umap'):
        _embedding_options.append(key)
    
palette = lambda n: [(int(r), int(g), int(b), a) for r, g, b, a in (plt.get_cmap(theme)(np.linspace(0, 1, n)) * [255, 255, 255, 1])]

COLOR_MAPS = {}
for key in _color_options:
    data = displayProperties[key]['data']
    if isinstance(data[0], str):
        COLOR_MAPS[key] = btr.factor_cmap(key, palette(len(list(set(data)))), list(set(data)))
    else:
        COLOR_MAPS[key] = btr.linear_cmap(key, palette(600), low=min(data), high=max(data))
        COLOR_MAPS[key+"_log"] = btr.log_cmap(key, palette(600), low=min(data), high=max(data))
        
_graphs = []
for y_data in zip_longest(*[displayProperties[key]['data'] for key in _to_graph]):
    img = BytesIO()
    fig = plt.figure(figsize=(2,1))
    for p, color in zip(y_data, cycle(['red', 'black', 'blue', 'green', 'purple', 'orange', 'brown', 'pink', 'gray', 'olive'])):
        plt.plot(p, color=color, linewidth=0.5)
    plt.axis('off')
    plt.savefig(img, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    _graphs.append('data:image/png;base64,' + base64.b64encode(img.getvalue()).decode())  
PLOT_DF['graph'] = _graphs

EMBEDDING_TITLES = {}
for key in _embedding_options:
    reducer = umap.UMAP(**displayProperties[key]['umap'])
    embedding = reducer.fit_transform(displayProperties[key]['data'])
    EMBEDDING_TITLES[key] = str(reducer)+" fit of "+key
    PLOT_DF[key+"_x"] = embedding[:,0]
    PLOT_DF[key+"_y"] = embedding[:,1]

UMAP(n_neighbors=10, verbose=True)
Thu Jul 13 03:36:38 2023 Construct fuzzy simplicial set
Thu Jul 13 03:36:38 2023 Finding Nearest Neighbors
Thu Jul 13 03:36:38 2023 Finished Nearest Neighbor Search
Thu Jul 13 03:36:38 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Thu Jul 13 03:36:42 2023 Finished embedding


Make plot

In [8]:
TOOLTIPS = f"""
    <div>
        {"<div>" + "".join([f'<div><span style="font-size: 15px; font-weight: bold;">{attr_key}: @{attr_key}</span></div>' for attr_key in _tooltips]) + "</div>"}
        {"<div><img src='@graph' style='margin: 5px 5px 5px 5px'/></div>" if any(_to_graph) else ""}
        <hr style="margin: 5px 5px 5px 5px"/><br>
    </div>
""" if any(_tooltips) else None
embeddingSelect = Select(
    title="Embedding",
    value=_embedding_options[0],
    options=_embedding_options
)
coloringSelect = Select(
    title="Color by",
    value=_color_options[0],
    options=_color_options
)
logCheck = Checkbox(
    label="Log color scale",
    active=False
)
fig = bk.figure(tools=(
    CopyTool(), 
    HoverTool(tooltips=TOOLTIPS), 
    LassoSelectTool()
))
plotPoints = fig.circle(
    x=f'{_embedding_options[0]}_x',
    y=f'{_embedding_options[0]}_y',
    source=PLOT_DF, 
    size=7,
    line_color='black',
    line_width=0.25,
    fill_alpha=0.7,
    fill_color=COLOR_MAPS[_color_options[0]],
)
colorbar = ColorBar(
    color_mapper=COLOR_MAPS[_color_options[0]].transform,
    padding=1
)
fig.add_layout(colorbar, 'right')
embeddingSelect.js_on_change('value', CustomJS(
    args=dict(plotPoints=plotPoints, fig=fig, EMBEDDING_TITLES=EMBEDDING_TITLES),
    code=" plotPoints.glyph.x = cb_obj.value + '_x'; plotPoints.glyph.y = cb_obj.value + '_y'; fig.title.text = EMBEDDING_TITLES[cb_obj.value]; "
))
coloringSelect.js_on_change('value', CustomJS(
    args=dict(plotPoints=plotPoints, color_maps=COLOR_MAPS, colorbar=colorbar, logCheck=logCheck),
    code="""
    const selectedAttr = cb_obj.value;
    
    if (logCheck.active) {
        plotPoints.glyph.fill_color = color_maps[selectedAttr + '_log'];
        colorbar.color_mapper = color_maps[selectedAttr + '_log']['transform'];
    } else {
        plotPoints.glyph.fill_color = color_maps[selectedAttr];
        colorbar.color_mapper = color_maps[selectedAttr]['transform'];
    }
    """
))
logCheck.js_on_change('active', CustomJS(
    args=dict(plotPoints=plotPoints, color_maps=COLOR_MAPS, colorbar=colorbar, coloringSelect=coloringSelect),
    code="""
    const selectedAttr = coloringSelect.value;
    
    if (cb_obj.active) {
        plotPoints.glyph.fill_color = color_maps[selectedAttr + '_log'];
        colorbar.color_mapper = color_maps[selectedAttr + '_log']['transform'];
    } else {
        plotPoints.glyph.fill_color = color_maps[selectedAttr];
        colorbar.color_mapper = color_maps[selectedAttr]['transform'];
    }
    """
))
fig.grid.visible = False
fig.axis.visible = False
plot = bkl.gridplot([[embeddingSelect, coloringSelect, logCheck], [fig]],toolbar_location='below')
bk.output_notebook()
bk.show(plot)
bk.save(plot, filename='test.html')

Loading BokehJS ...

/tmp/ipykernel_15657/4171922877.py:79: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  bk.save(plot, filename='test.html')
/tmp/ipykernel_15657/4171922877.py:79: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  bk.save(plot, filename='test.html')


'/home/guutz/dawn/test.html'